In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
sensorDf = pd.read_csv('../suzhou/data/人工匹配卡口数据-V2.csv')
#sensorDf['ID'][:50]
sensorDf['device_id'][:50]

0     32050617041320001291
1     32050000001320002392
2     32050000001320001229
3     32050000001320001233
4     32050204041320002359
5     32050204041320002365
6     32050204041320002371
7     32050204041320002355
8     32050204041320002375
9     32050000001320002533
10    32050617041320002020
11    32050617041320002234
12    32050617041320002233
13    32050000001320002537
14    32050000001320002535
15    32050000001320002529
16    32058117881320001953
17    32058117881320002299
18    32058117881320002395
19    32058117881320000454
20    32058117881320002437
21    32058117881320000988
22    32058117881320000471
23    32058117881320002005
24    32058117881320002385
25    32058200001310187873
26    32058117881320002172
27    32058117881320002117
28    32050202041320001820
29    32050202041320002260
30    32050202041320002237
31    32050202041320002235
32    32050202041320002028
33    32050202041320002026
34    32050204041320001632
35    32050204041320001630
36    32050204041320001616
3

In [4]:
#路网数据
originRoad = gpd.read_file("../suzhou/data/原始数据/road/苏州.shp")
originRoad = originRoad[['ID', 'KIND', 'WIDTH', 'DIRECTION', 'FUNCCLASS', 'LENGTH', 'SPEEDCLASS', 'PHYLANES2E', 'PHYLANEE2S', 'geometry']]
print(originRoad.shape)
print(type(originRoad))
#查找originalRoad中ID字段为1的数据

(421553, 10)
<class 'geopandas.geodataframe.GeoDataFrame'>


,ID,KIND,WIDTH,DIRECTION,FUNCCLASS,LENGTH,SPEEDCLASS,PHYLANES2E,PHYLANEE2S,geometry


In [10]:
print(originRoad.duplicated().sum())  # 检查重复行的数量

0


In [9]:
print(originRoad.shape)
#将originRoad中所有字段写入CSV文件
originRoad.to_csv("../suzhou/data/苏州_路网数据.csv", index=False, encoding='utf-8')

(421553, 10)


In [18]:
originRoadCsv = pd.read_csv("../suzhou/data/苏州_路网数据.csv")
originRoadCsv[originRoad['ID'] == "4158550"]
#查看每列的数据类型
print(originRoadCsv.dtypes)
print(sensorDf.dtypes)

ID              int64
KIND           object
WIDTH           int64
DIRECTION       int64
FUNCCLASS       int64
LENGTH        float64
SPEEDCLASS      int64
PHYLANES2E      int64
PHYLANEE2S      int64
geometry       object
dtype: object
device_id     object
num            int64
location      object
ID           float64
dtype: object


In [20]:
#将路网数据和卡口流量数据进行合并
# 将 ID 字段统一为同一类型
originRoadCsv['ID'] = originRoadCsv['ID'].astype(float)  # 将 originRoadCsv 的 ID 转为 float
sensorDf['ID'] = sensorDf['ID'].astype(float)  # 将 sensorDf 的 ID 转为 float
# 以 ID 为中介，将 sensorDf 的 deviceId 列拼接到 originRoadCsv 后
originRoadCsv = pd.merge(originRoadCsv,  sensorDf[['ID', 'device_id']], on='ID', how='left')
# 保存结果到新文件（可选）
originRoadCsv.to_csv("../suzhou/data/苏州_路网数据_带卡口.csv", index=False)

In [22]:
firstProcessRoadCsv = pd.read_csv("../suzhou/data/苏州_路网数据_带卡口.csv")
firstProcessRoadCsv[firstProcessRoadCsv['ID'] == 4158550]

,ID,KIND,WIDTH,DIRECTION,FUNCCLASS,LENGTH,SPEEDCLASS,PHYLANES2E,PHYLANEE2S,geometry,device_id
164559,4158550.0,0601,55,1,5,0.109,6,1,1,"LINESTRING Z (120.52720999999998 31.16519 0, 1...",32050000001320002392


In [24]:
firstProcessRoadCsv[firstProcessRoadCsv['device_id'] == "32050000001320002392"]

,ID,KIND,WIDTH,DIRECTION,FUNCCLASS,LENGTH,SPEEDCLASS,PHYLANES2E,PHYLANEE2S,geometry,device_id
164559,4158550.0,0601,55,1,5,0.109,6,1,1,"LINESTRING Z (120.52720999999998 31.16519 0, 1...",32050000001320002392


In [30]:
volume = pd.read_csv("../suzhou/2021/09-01/00.csv")
print(volume[volume['DeviceID']=="123456789123456789"])
print(volume.dtypes)

             DeviceID  Quantity
0  123456789123456789       120


ValueError: could not convert string to float: 'EjIFBQAAABMgIA=='

In [34]:
#进行匹配，将卡口流量数据和路网数据进行匹配
import os
import pandas as pd

# 根文件夹路径
root_folder = "../suzhou/2021"


# 循环读取根文件夹下的所有子文件夹
for sub_folder in os.listdir(root_folder):
    sub_folder_path = os.path.join(root_folder,  sub_folder)  # 获取子文件夹完整路径
    # 循环读取子文件夹下的所有 CSV 文件
    for file_name in os.listdir(sub_folder_path):
        if file_name.endswith(".csv"):   # 确保文件是 CSV 文件
            file_path = os.path.join(sub_folder_path,  file_name)  # 获取文件完整路径
            df = pd.read_csv(file_path)   # 读取 CSV 文件
            merged_df = pd.merge(df,  firstProcessRoadCsv, left_on='DeviceID', right_on='device_id', how='left')
            merged_df.to_csv(file_path,  index=False)

In [35]:
import os
import pandas as pd

# 根文件夹路径
root_folder = "../suzhou/2022"


# 循环读取根文件夹下的所有子文件夹
for sub_folder in os.listdir(root_folder):
    sub_folder_path = os.path.join(root_folder,  sub_folder)  # 获取子文件夹完整路径
    # 循环读取子文件夹下的所有 CSV 文件
    for file_name in os.listdir(sub_folder_path):
        if file_name.endswith(".csv"):   # 确保文件是 CSV 文件
            file_path = os.path.join(sub_folder_path,  file_name)  # 获取文件完整路径
            df = pd.read_csv(file_path)   # 读取 CSV 文件
            merged_df = pd.merge(df,  firstProcessRoadCsv, left_on='DeviceID', right_on='device_id', how='left')
            merged_df.to_csv(file_path,  index=False)